In [1]:
from typing import Callable, Tuple, Sequence

from push4.expression import LiteralExpression, RegisterExpression, expression_from_function
import push4.interpreter as push

# Push4 - (Name TBD)

The framework changes the way Push is used for program synthesis. Instead of using a Push interpreter to execute programs, it uses Push to compile a linear (or nested) structure of `Expressions` into a DAG/tree that serves a the program/function that was evoled.

The key features are:
- No need to specify any atom generators, instruction sets, or Push types.
- Evolving program that use the functions/types defined in an existing codebase. This includes collection types.
- Reusing programs is much faster because the Push language is only used during evolution.
- **The evolved programs can be converted into source code of the host language!**

## Some Pure Functions

This version of Push does not use a pre-defined set of Types, Stacks, or Instructions. Instead, a set of functions **with type hints** is supplied and the necessary typing information is captured via reflection.

These must be pure fnctions. No side effects. No updating mutable sta|te.

In [2]:
def int_add(a: int, b: int) -> int:
    return a + b


def int_mult(a: int, b: int) -> int:
    return a * b


def str_repeat(s: str, n: int) -> str:
    return s * n


def str_split(s: str, on: str) -> Sequence[str]:
    return s.split(on)


def first(seq: Sequence[str]) -> str:
    return seq[0]

If we call `int_add` from within our own funciton, we would need to supply `a` and `b`. They could come from constants, our function's parameters, or the result of calling another function with returns an `int`.

We will represent our program as a tree of `Expressions`. An `Expression` is a wrapped funciton or literal with an explicit return type. To safely generate a valid `Expression` tree from a linear genome, we will use Push.

## Genome of Anything

Our genomes is a linear sequence of objects. Every gene is translated into an `Expression`. Expressions are a wrapper for some code (function or literal) which denotes the data type that will be returned by the underlying code. To determine how what kind of expression each gene should be, we only need to know the gene's type (ie. function, tuple, something else).

The `(int, str, type)` tuples seen in the genome are a hack to account for the fact that Python does not have first class symbols. They represent the position and name of a symbol that will be injected from the inputs to our final program. 

The genome below corresponds to a program with the following signature:

```py
def repeat_first_word(text: str, times: int) -> str:
    ...
```

The program should split on a space character and return a string with the first word repeated some number of times.
For example: `("Hello World", 3)` should yeild `"HelloHelloHello"`.

In [3]:
genome = [
    " ",
    (0, "text", str),  # Arg 0 is called "text" and is a str.
    str_split,
    False,             # Useless literal. Push will handle it.
    first,
    (1, "times", int), # Arg 1 is called "times" and is an int.
    str_repeat,
    int_add,           # Useless function. Push will handle it.
]

## Push Into A Program

Unlike other Push implementations, this version uses the Push execution model to compile a DAG/tree which can be 1) executed as a program, 2) printed out as source code, and 3) possibly optimized using some kind of algebra.

The execution loop is as follows:

  1. Let `G` be the next gene in the genome.
  2. Convert `G` into an expression, `E`, based on its type.
  3. If `E` doesn't require inputs (aka is a literal or input symbol)
    1. Wrap `E` in a leaf node and put on the stack corresponding to the expressions output data type.
  4. If `E` requires inputs (aka a "function" expression)
    1. Let `T` be vector of types required by arguments of `E`.
    2. If there are insuffient nodes from the stacks corresponding to the types in `T`...
      1. Discard `E`
    3. Otherwise...
      1. Pop the nodes from the stacks corresponding to each evlement of `T`.
      2. Wrap `E` in a non-leaf node and use the popped nodes as its children.
      3. Push the node (and children) to the stack corresponding to `E`'s output type as a sin|gle element.
      
After all genes have been "compiled", the one Node can be popped from a stack and used as our program. We choose the stack that corresponds to the data type which we want our program to produce.

Below is a Push trace of proccessing the above genome. You can see that the Push state starts off with no stacks. Stacks are added as new `Expression` return types are encountered. Also, notice how collection types don't need to be treated any differently than atomic types.

Step 2 applies evaluated the `str_split(str,str)->Sequence[str]` Expression. The expression require 2 str arguments. There are already 2 Expression Nodes that return sstrings on the `str` stack. Those nodes are consumed and used as the children to the `str_split` expression, which is placed on the `Sequence[str]` stack (see step 3).

After Push evaluation, we take ask the interpreter/compiler to pop the top Node off the `Expression[String]` stack and convert it into an `Expression[Tree]` that acts as our program.

In [4]:
program = push.run(genome, str)

0 - Evaluating ' ':str

1 - Evaluating symbol:text:str
str : [Node<' ':str>]

2 - Evaluating str_split(str,str)->Sequence[str]
str : [Node<symbol:text:str>, Node<' ':str>]

3 - Evaluating False:bool
str : []
Sequence[str] : [Node<str_split(str,str)->Sequence[str]>]

4 - Evaluating first(Sequence[str])->str
str : []
Sequence[str] : [Node<str_split(str,str)->Sequence[str]>]
bool : [Node<False:bool>]

5 - Evaluating symbol:times:int
str : [Node<first(Sequence[str])->str>]
Sequence[str] : []
bool : [Node<False:bool>]

6 - Evaluating str_repeat(str,int)->str
str : [Node<first(Sequence[str])->str>]
Sequence[str] : []
bool : [Node<False:bool>]
int : [Node<symbol:times:int>]

7 - Evaluating int_add(int,int)->int
str : [Node<str_repeat(str,int)->str>]
Sequence[str] : []
bool : [Node<False:bool>]
int : []



We can visualize the program as a tree. This is much easier for humans to understand than a traditional Push program.

In [5]:
program.pprint()

 - str_repeat(str,int):str
 |  - first(Sequence[str]):str
 |  |  - str_split(str,str):Sequence[str]
 |  |  |  - symbol:text:str
 |  |  |  - ' ':str
 |  - symbol:times:int


Let's test out our program. Recall the signature was `(text: str, times: int) -> str`.

This does not require Push, and should be much faster to execute. 

In [6]:
program.eval("Hello World", 2)

'HelloHello'

In [7]:
program.eval("Hi", 3)

'HiHiHi'

### Generating Python Source Code

The program seems to be working, but there's more! If we give the program a name and some argument names we can generate source code! 

In [8]:
code = program.to_code(
    function_name="repeat_first_word", 
    args=[("text", str), ("times", int)]
)
print(code)

def repeat_first_word(text: str, times: int) -> str:
    return str_repeat(first(str_split(text, ' ')), times)


## Test out the generated code

In [9]:
def repeat_first_word(text: str, times: int) -> str:
    return str_repeat(first(str_split(text, ' ')), times)

In [10]:
repeat_first_word("Hello World", 2)

'HelloHello'

In [11]:
repeat_first_word("Hi", 3)

'HiHiHi'

## Some Commens and Concerns

#### Less control over behavior

We no long control the behavior of the "atomic" code units.  Clojush controls instruction implemenations. Unless we implement safe functions to use in genomes, doing evolution this way will re-introduce the possibility of runtime errors.

#### What about the exec?

In this system, we don't have the ability to use the exec stack for control flow because Push is only building the program. To maintain control flow in our programs, we would need to capture it in the DAG (:barf:) or in the functions. This might not be so bad with the addition of collections and higher order functions.

The exec stack might still have a use. Including iteration in the Push evaluation would create redundancy in the DAG/tree which might help evolution.

